<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/multitaxa/multitaxa_train_tf_ssd_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Tensorflow Faster-RCNN and SSD models to detect butterflies and moths (Lepidoptera) from EOL images
---   
*Last Updated 16 March 2020*  
Use images and bounding box coordinates to train Faster-RCNN and SSD Object Detection Models implemented in Tensorflow to detect butterflies and moths from EOL images.

Datasets exported from [lepidoptera_preprocessing.ipynb](https://github.com/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/lepidoptera/lepidoptera_preprocessing.ipynb) were already downloaded to Google Drive in lepidoptera_preprocessing.ipynb. 

Notes:   
* For each 24 hour period on Google Colab, you have up to 12 hours of GPU access. Training the object detection model on bats took 30 hours split into 3 days.

* Make sure to set the runtime to Python 2 with GPU Hardware Accelerator.  

References:   
* [Official Tensorflow Object Detection API Instructions](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html)   
* [Medium Blog on training using Tensorflow Object Detection API in Colab](https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b)

## Installs (run this every time)
---
Install the Tensorflow Object Detection API directly to this Colab notebook.

In [0]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Download, compile and build all ingredients for the Tensorflow Object Detection API
# These steps take a couple of minutes and print a lot of output

# Make a working directory train/ in your Google Drive and include the path here (all paths in other sections stay the same)
import os
%cd drive/My Drive/fall19_smithsonian_informatics/train
if not os.path.exists("tf_models"):
  !mkdir tf_models
%cd tf_models

# Install Tensorflow Object Detection API
import pathlib
if not pathlib.Path("models").exists():
  !git clone https://github.com/tensorflow/models.git

# Clone the COCO API repository to your Google Drive
if not pathlib.Path("pycocotools").exists():
  !git clone https://github.com/cocodataset/cocoapi.git
  # Move needed folders to tf_models/pycocotools and delete remaining contents of cocoapi/ to save space
  !cd cocoapi/PythonAPI; make; cp -r pycocotools ../..
  !rm -rf cocoapi

# Install libraries
!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

# Compile object detection api using Google Protobuf
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

# Update system path variables
os.environ['PYTHONPATH'] = ':/drive/My Drive/fall19_smithsonian_informatics/train/tf_models/models/research/:/drive/My Drive/fall19_smithsonian_informatics/train/tf_models/models/research/slim/'
!echo $PYTHONPATH

import sys
print(sys.path)
sys.path.append("/usr/local/lib/python2.7/dist-packages/tensorflow/contrib/slim")
print(sys.path)

# Build slim
!python slim/setup.py build
!python slim/setup.py install

# Copy slim to specified directories to avoid errors in model_builder_test.py
!cp -R slim/ /usr/local/lib/python2.7/dist-packages/object_detection-0.1-py2.7.egg/
if not os.path.exists("object_detection/slim/nets"):
  !cp -R slim/nets/ object_detection/

# Test build of model
!python object_detection/builders/model_builder_test.py

In [0]:
#%cd drive/My Drive/fall19_smithsonian_informatics/train
#!unzip tf_models/models-20200216T051538Z-001.zip

## Model preparation (only need to run these once)
---

### Download and extract pre-trained model   

In [0]:
# Download pre-trained model from Tensorflow Object Detection Model Zoo
# SSD and Faster-RCNN both included as options below
# modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb

# cd to train/
%cd ../../..

import os
import shutil
import glob
import urllib
import tarfile

# Make folders for your training files for each model
# RCNN Model
if not (os.path.exists('tf_models/train_demo')):
  !mkdir tf_models/train_demo
if not (os.path.exists('tf_models/train_demo/rcnn')):
  !mkdir tf_models/train_demo/rcnn
if not (os.path.exists('tf_models/train_demo/rcnn/pretrained_model')):
  !mkdir tf_models/train_demo/rcnn/pretrained_model
if not (os.path.exists('tf_models/train_demo/rcnn/finetuned_model')):
  !mkdir tf_models/train_demo/rcnn/finetuned_model
if not (os.path.exists('tf_models/train_demo/rcnn/trained')):
  !mkdir tf_models/train_demo/rcnn/trained
# Download the model
MODEL = 'faster_rcnn_resnet50_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'tf_models/train_demo/rcnn/pretrained_model'
if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

# SSD Model
if not (os.path.exists('tf_models/train_demo/ssd')):
  !mkdir tf_models/train_demo/ssd
if not (os.path.exists('tf_models/train_demo/ssd/pretrained_model')):
  !mkdir tf_models/train_demo/ssd/pretrained_model
if not (os.path.exists('tf_models/train_demo/ssd/finetuned_model')):
  !mkdir tf_models/train_demo/ssd/finetuned_model
if not (os.path.exists('tf_models/train_demo/ssd/trained')):
  !mkdir tf_models/train_demo/ssd/trained
# Download the model
MODEL = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'tf_models/train_demo/ssd/pretrained_model'
if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

### Convert training data to tf.record format

In [0]:
# Upload lepidoptera_generate_tfrecord.py to your Google Drive
!gdown --id 1neN-JupseY4ug2k7MU7s6CJr888Czp-8 

In [0]:
# Convert crops_test to tf.record format for test data
# Modified from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
!python lepidoptera_generate_tfrecord.py --csv_input='/content/drive/My Drive/fall19_smithsonian_informatics/train/lepidoptera_crops_test_notaug_fin.csv'  --output_path= "/content/drive/My Drive/fall19_smithsonian_informatics/train/test_images/tf.record"  --image_dir="/content/drive/My Drive/fall19_smithsonian_informatics/train/test_images"

In [0]:
!mv tf.record test_images/

In [0]:
# Convert crops_train to tf.record format for train data
# Modified from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
!python lepidoptera_generate_tfrecord.py --csv_input='/content/drive/My Drive/fall19_smithsonian_informatics/train/lepidoptera_crops_train_aug_fin.csv'  --output_path= "/content/drive/My Drive/fall19_smithsonian_informatics/train/images/tf.record"  --image_dir="/content/drive/My Drive/fall19_smithsonian_informatics/train/images"

In [0]:
# Upload label map for class Lepidoptera to your Google Drive
!gdown --id 1QrrSKLm15UQTfAd-IDm--LQJ75f6nb3V

### Upload modified config files for Faster-RCNN and SSD models to your Google Drive
If you have errors with training, check the pipline_config_path and model_dir in the config files for SSD or Faster-RCNN model

In [0]:
%cd tf_models/models/research/object_detection/samples/configs
# SSD ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03_bats.config 
!gdown --id 192N8V3o-4z3vJRNMSgusl3vgwXH04rtX
# Faster-RCNN faster_rcnn_resnet50_coco_2018_01_28_bats.config
!gdown --id 18wlrqTWKLZmcvaSsNbdLGM98lFlC8CB6

# cd back to train
%cd ../../../../../..

## Train the model
--- 
To switch between training Faster-RCNN and SSD models, see notes at the beginning of each code block on which lines to comment out and move to the bottom of the cell.

In [0]:
# First time running this notebook: cd to train after running Model Preparation steps above
#%cd train
# Subsequent times running this notebook: cd to train
%cd ../../..

In [0]:
# Actual training
# Modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb
# Change pipline_config_path and model_dir to the appropriate config file for SSD or Faster-RCNN model
import matplotlib
matplotlib.use('Agg')

!python tf_models/models/research/object_detection/model_main.py \
    --alsologtostderr \
    --num_train_steps=200000 \
    --num_eval_steps=500 \
    --pipeline_config_path=tf_models/models/research/object_detection/samples/configs/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03_bats.config \
    --model_dir=tf_models/train_demo/ssd/trained/ \
    #--pipeline_config_path=tf_models/models/research/object_detection/samples/configs/faster_rcnn_resnet50_coco_2018_01_28_bats.config \
    #--model_dir=tf_models/train_demo/rcnn/trained/ \

In [0]:
# Export trained model
# Modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb
# Change os.listdir, pipeline_config_path, output_directory and trained_checkpoint_prefix when switching between SSD and Faster-RCNN models
%cd train

lst = os.listdir('tf_models/train_demo/ssd/trained')
#lst = os.listdir('tf_models/train_demo/rcnn/trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python tf_models/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=tf_models/models/research/object_detection/samples/configs/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03_bats.config \
    --output_directory=tf_models/train_demo/ssd/finetuned_model \
    --trained_checkpoint_prefix=tf_models/train_demo/ssd/trained/$last_model \
    #--pipeline_config_path=tf_models/models/research/object_detection/samples/configs/faster_rcnn_resnet50_coco_2018_01_28_bats.config \
    #--output_directory=tf_models/train_demo/rcnn/finetuned_model \
    #--trained_checkpoint_prefix=tf_models/train_demo/rcnn/trained/$last_model \

### When finished training

In [0]:
# Evaluate trained model to get mAP and IoU stats
# Change pipeline_config_path and checkpoint_dir when switching between SSD and Faster-RCNN models
import matplotlib
matplotlib.use('Agg')

!python tf_models/models/research/object_detection/model_main.py \
    --alsologtostderr \
    --pipeline_config_path=tf_models/models/research/object_detection/samples/configs/faster_rcnn_resnet50_coco_2018_01_28_bats.config \
    --checkpoint_dir=tf_models/train_demo/rcnn/trained/ \
    #--pipeline_config_path=tf_models/models/research/object_detection/samples/configs/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03_bats.config \
    #--checkpoint_dir=tf_models/train_demo/ssd/trained/ \ 

## Run test images through the trained object detector
---
Test image detection boxes are only needed for calculating mAP (mean average precision, a performance measure to compare models) and not for cropping. The functions below will only display resulting detection boxes on test images for visualization, but does not save their coordinates to a spreadsheet. 

### Imports

In [0]:
# cd to train
%cd ../../..
#%cd train

%tensorflow_version 1.0

import tensorflow as tf 
tf.compat.v1.enable_eager_execution()

# For importing/exporting files, working with arrays, etc
import os
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import numpy as np 
import csv
import matplotlib
import time
import pandas as pd

# For downloading the images
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
from collections import defaultdict
from io import StringIO

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import cv2

from IPython.display import display

sys.path.append("tf_models/models/research/")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
%matplotlib inline

### Prepare object detection functions and settings

In [0]:
# Change PATH_TO_CKPT below when switching between SSD and Faster-RCNN models
%cd train
# SSD Model
#PATH_TO_CKPT = 'tf_models/train_demo/ssd/finetuned_model' + '/frozen_inference_graph.pb'
# Faster RCNN Model
PATH_TO_CKPT = 'tf_models/train_demo/rcnn/finetuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'labelmap.pbtxt'
NUM_CLASSES = 1
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# For loading images into computer-readable format
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Function for loading images from urls
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image

# Can change min_score_thresh and max_boxes_to_draw
def show_inference(image_np_expanded):
  with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
      # Definite input and output Tensors for detection_graph
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      #max_boxes_to_draw = detection_boxes.shape[0] # add this line and remove (i) and (ii) below to show multiple detection boxes
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
      detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      #min_score_thresh = .7

      # Actual detection
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      
      # Visualization of the results of a detection
      # Modified from https://github.com/tensorflow/models/issues/4682
      im_height, im_width, im_depth = image_np.shape
      ymin = int((boxes[0][0][0]*im_height))
      xmin = int((boxes[0][0][1]*im_width))
      ymax = int((boxes[0][0][2]*im_height))
      xmax = int((boxes[0][0][3]*im_width))
      newImage = np.copy(image_np)
      newImage = cv2.rectangle(newImage, (xmin, ymax), (xmax, ymin), (255, 0, 157), 3)
      # Add labels to boxes
      #newImage = cv2.putText(newImage, label, (xmin, ymax-5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (153, 255, 255), 5, cv2.LINE_AA)

      return newImage

### Run test images through object detector

In [0]:
# Update path to your test images
PATH_TO_TEST_IMAGES_DIR = 'test_images/'
PATH_TO_OUT_IMAGES_DIR = 'test_images/out/'
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]
OUT_IMAGE_PATHS = [os.path.join(PATH_TO_OUT_IMAGES_DIR, name) for name in names]

# Loops through first 5 image urls from the text file
for im_num, im_path in enumerate(TEST_IMAGE_PATHS[:50], start=1):
 
    # Load in image
    image = Image.open(im_path)
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Record inference time
    start_time = time.time()
    # Detection and draw boxes on image
    show_inference(image_np_expanded)
    end_time = time.time()
    # Display progress message after each image
    print('Detection complete in {} of 145 test images'.format(im_num))

    # Plot and show detection boxes on images
    # If running detection on >50 images, comment out this portion
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(show_inference(image_np_expanded))
    plt.title('{}) Inference time: {}'.format(im_num, format(end_time-start_time, '.2f')))

    # Optional: Save image with detection boxes to Google Drive
    #img = Image.fromarray(show_inference(image_np_expanded))
    #img.save(OUT_IMAGE_PATHS[im_num-1])

### Get inference info for test images to compare object detection model times for YOLO, SSD, and Faster-RCNN

In [0]:
from PIL import Image
import os

# For exporting inference times
inf_time = []
img_path = []
im_dims = []

# Update path to your test images
PATH_TO_TEST_IMAGES_DIR = 'test_images/'
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]

# Loops through first 5 image urls from the text file
#for im_num, im_path in enumerate(TEST_IMAGE_PATHS[:5], start=1):
for im_num, im_path in enumerate(TEST_IMAGE_PATHS, start=1):
 
    # Load in image
    image = Image.open(im_path)
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Record inference time
    start_time = time.time()
    # Detection and draw boxes on image
    show_inference(image_np_expanded)
    end_time = time.time()
    # Display progress message after each image
    print('Detection complete in {} of 145 test images'.format(im_num))

    # Record inference time, image name and image dimensions to export
    inf_time.append(end_time-start_time)
    img_path.append(im_path)
    im_dims.append(image_np.shape)
    
inf_times = pd.DataFrame(([inf_time, img_path, im_dims]))
inf_times = inf_times.transpose()
#inf_times.to_csv("lepidoptera_inference_times_rcnn.csv", index=False, header=("time (sec)", "filepath", "image_dims (h, w, d)"))
inf_times.to_csv("lepidoptera_inference_times_ssd.csv", index=False, header=("time (sec)", "filepath", "image_dims (h, w, d)"))
print(inf_times.head())

### Run other images (from individual URLs) through object detector

In [0]:
# Test trained model on test images
from PIL import Image

# Put your urls here
image_urls = ["https://upload.wikimedia.org/wikipedia/commons/b/be/Batman_%28retouched%29.jpg",
              "https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Bela_Lugosi_as_Dracula%2C_anonymous_photograph_from_1931%2C_Universal_Studios.jpg/690px-Bela_Lugosi_as_Dracula%2C_anonymous_photograph_from_1931%2C_Universal_Studios.jpg"]

# Loops through image_urls
for im_num, image_url in enumerate(image_urls, start=1):
  try:
    # Load in image
    image_np = url_to_image(image_url)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Record inference time
    start_time = time.time()
    # Detection and draw boxes on image
    show_inference(image_np_expanded)
    end_time = time.time()
    # Display progress message after each image
    print('Detection complete in {} of 2 images'.format(im_num))

    # Plot and show detection boxes on images
    # If running detection on >50 images, comment out this portion
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(show_inference(image_np_expanded))
    plt.title('{}) Inference time: {}'.format(im_num, format(end_time-start_time, '.2f')))

  except:
    print('Check if URL from {} is valid'.format(image_url))

## Run EOL image bundles through the trained object detector & save results for cropping
---
Display resulting detection boxes on images and save their coordinates to chiroptera_det_crops.tsv for use cropping EOL images.

In [0]:
# Change PATH_TO_CKPT below when switching between SSD and Faster-RCNN models
%cd train
# SSD Model
#PATH_TO_CKPT = 'tf_models/train_demo/ssd/finetuned_model' + '/frozen_inference_graph.pb'
# Faster RCNN Model
PATH_TO_CKPT = 'tf_models/train_demo/rcnn/finetuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'labelmap.pbtxt'
NUM_CLASSES = 1
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# For loading images into computer-readable format
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Function for loading images from urls
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image

# Can change min_score_thresh and max_boxes_to_draw
def show_inference(image_np_expanded):
  with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
      # Definite input and output Tensors for detection_graph
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represents the level of confidence for each of the objects.
      detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
      detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Optional: adjust score confidence threshold for display
      #min_score_thresh = .7

      # Actual detection
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      
      # Visualization of the results of a detection
      # Modified from https://github.com/tensorflow/models/issues/4682
      im_height, im_width, im_depth = image_np.shape
      ymin = int((boxes[0][0][0]*im_height))
      xmin = int((boxes[0][0][1]*im_width))
      ymax = int((boxes[0][0][2]*im_height))
      xmax = int((boxes[0][0][3]*im_width))
      newImage = np.copy(image_np)
      newImage = cv2.rectangle(newImage, (xmin, ymax), (xmax, ymin), (255, 0, 157), 3)
      # Add labels to boxes
      #newImage = cv2.putText(newImage, label, (xmin, ymax-5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (153, 255, 255), 5, cv2.LINE_AA)

      # Export bounding boxes to drive
      with open('/content/drive/My Drive/fall19_smithsonian_informatics/train/lepidoptera_det_crops_20000_a.tsv', 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  crop_width = xmax-xmin
                  crop_height = ymax-ymin
                  tsv_writer.writerow([image_url, im_height, im_width, 
                            xmin, ymin, xmax, ymax])
      return newImage

In [0]:
# Use URLs from EOL image URL bundles
# Comment out to use either 1000 or 20000 image bundles
#1000 images
#urls = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Lepidoptera_breakdown_download_000001.txt'
#20000 images
urls = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Lepidoptera_20K_breakdown_download_000001.txt'

df = pd.read_csv(urls)
df.columns = ["link"]
pd.DataFrame.head(df)

In [0]:
# Write header row of output crops file
with open('/content/drive/My Drive/fall19_smithsonian_informatics/train/lepidoptera_det_crops_20000_a.tsv', 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow(["image_url", "im_height", "im_width", 
                            "xmin", "ymin", "xmax", "ymax"])

In [0]:
# Test trained model on test images
from PIL import Image

# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# Loops through first 5 image urls from the text file
#for i, row in df.head(5).itertuples(index=True, name='Pandas'):

# For ranges of rows or all rows, use df.iloc
# Can be useful if running detection in batches
for i, row in df.iloc[:5000].iterrows():

  try:
    # Load in image
    image_url = df.get_value(i, "link")
    image_np = url_to_image(image_url)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Record inference time
    start_time = time.time()
    # Detection and draw boxes on image
    show_inference(image_np_expanded)
    end_time = time.time()
    # Display progress message after each image
    print('Detection complete in {} of 20000 test images'.format(i+1))

    # Plot and show detection boxes on images
    # If running detection on >50 images, comment out this portion
    #_, ax = plt.subplots(figsize=(10, 10))
    #ax.imshow(show_inference(image_np_expanded))
    #plt.title('{}) Inference time: {}'.format(i+1, format(end_time-start_time, '.2f')))

  except:
    print('Check if URL from {} is valid'.format(image_url))